In [38]:
from tsdb_client import TSDBClient
import pandas as pd
import numpy as np
import trading_calendars as tc
import pytz
xtai = tc.get_calendar("XTAI")
import datetime


In [27]:
order = pd.read_csv('status/Order.txt', header = None, names = ['trader_id','order_id', 'security_type', 'order_time', 'code', 'order_type', 'action', 'order_qty', 'order_price', 'status'], dtype='str')
deal = pd.read_csv('status/Deal.txt', header = None, names = ['trader_id','order_id', 'security_type', 'trade_time', 'code', 'order_type', 'action', 'qty', 'price', 'status'], dtype='str')

In [3]:
def time_transform(time):
    return datetime.datetime.strptime(time, '%H%M%S').time()

def security_type_transform(security_type):
    if security_type == "現股":
        return 'S'
    elif security_type == "期貨":
        return 'F'

def action_transform(action):
    if action == 'Buy': return 'B'
    if action == 'Sell': return 'S'
        

In [29]:

deal['security_type'] = deal['security_type'].apply(security_type_transform)
deal['trade_time'] = deal['trade_time'].apply(time_transform)
deal['trade_date'] = datetime.datetime.now().date() 
deal['action'] = deal['action'].apply(action_transform)


KeyError: 'strategy'

In [28]:
order['security_type'] = order['security_type'].apply(security_type_transform)
order['order_time'] = order['order_time'].apply(time_transform)
order['order_date'] = datetime.datetime.now().date()
order['action'] = order['action'].apply(action_transform)



In [39]:

from tsdb_client import TSDBClient
from utils.Config import EnvConfig

cli = TSDBClient(
    host=EnvConfig.DB_HOST,
    port=EnvConfig.DB_PORT,
    user=EnvConfig.DB_USER,
    password=EnvConfig.DB_PASSWORD,
    db=EnvConfig.DB_DATABASE,
)


In [6]:
strategy_df = cli.execute_query('''
SELECT * FROM dealer.strategy
''', 
    out_type='df')

In [7]:

def strategy_align(log_type: str, strategy_name: str, order: pd.DataFrame):

    log = pd.read_csv(f'print/{strategy_name}/{log_type}.log', header = None, names = ['serial', 'security_type', 'timestamp', 'code', 'order_type', 'action', 'order_qty', 'order_price'])

    for index, row in log.iterrows():
    
        cur_order = order[(order['security_type'] == row.security_type[0]) & (order['code'] == str(row.code)) & (order['order_type'] == row.order_type) & 
        (order['action'] == row.action) & (order['order_price'].astype('float') == row.order_price)]
        
        for index in cur_order.index:
            order.loc[index, 'strategy'] = int(strategy_df[strategy_df['name'] == strategy_name]['id'].values[0])
    return order


In [8]:
def save2db(df: pd.DataFrame, table: str):
    cli = TSDBClient(
        host=EnvConfig.DB_HOST,
        port=EnvConfig.DB_PORT,
        user=EnvConfig.DB_USER,
        password=EnvConfig.DB_PASSWORD,
        db=EnvConfig.DB_DATABASE,
    )
    if table == "dealer.orders":
        date_ = "order_date"
    else:
        date_ = "trade_date"

    

    max_date: dt.datetime = cli.execute_query(f"select max({date_}) from {table}")[0][0]
    # only insert new data
    if max_date is not None:
        cond = pd.to_datetime(df[date_]) >= pd.to_datetime(max_date)
        df = df.loc[cond]
    

    result = cli.execute_values_df(df, table)
    if result == 1:
        raise Exception("save2db error")
        

In [9]:
dealer_trades = cli.execute_query(f"select * from dealer.trades", out_type='df')

In [10]:
append_index_list = []

for index, row in deal.iterrows():
    cur_dealer_trades = dealer_trades[(dealer_trades['order_id'] == row.order_id) & (dealer_trades['code'] == row.code)]
    if len(cur_dealer_trades) == 0:
        append_index_list.append(index)


In [54]:
cli.execute_query("select * from dealer.ft_get_positions_fifo(CURRENT_DATE, 'B') union select * from dealer.ft_get_positions_fifo(CURRENT_DATE, 'S')", out_type='df')

,tdate,strategy,code,action,qty,cost_amt,avg_prc,first_entry_date
0,2023-05-29,2,8048,B,3,148350.0,49.45,2023-05-18
1,2023-05-29,1,2882,B,12,522000.0,43.50,2023-05-16
2,2023-05-29,1,4129,B,8,499600.0,62.45,2023-05-26
3,2023-05-29,1,2836,B,3,37200.0,12.40,2023-05-25
4,2023-05-29,2,8446,B,2,228500.0,114.25,2023-05-18
5,2023-05-29,1,4958,B,2,227000.0,113.50,2023-05-22


In [33]:
deal.iloc[append_index_list]

,trader_id,order_id,security_type,trade_time,code,order_type,action,qty,price,status,strategy,trade_date
4,025,W003V,S,10:28:27,8446,ROD,B,1,113.5,NaN,0,2023-05-19


In [282]:
dealer_trades

,id,trader_id,broker_id,account_id,strategy,order_id,order_type,seqno,security_type,trade_date,trade_time,code,action,price,qty
0,72,025,None,None,1,W0042,ROD,100000016058,S,2023-05-16,09:00:07,2882,B,43.50,1
1,73,025,None,None,1,W0042,ROD,100000016059,S,2023-05-16,09:00:07,2882,B,43.50,1
2,74,025,None,None,1,W0042,ROD,100000016060,S,2023-05-16,09:00:07,2882,B,43.50,1
3,75,025,None,None,1,W0042,ROD,100000016061,S,2023-05-16,09:00:07,2882,B,43.50,1
4,76,025,None,None,1,W0042,ROD,100000016057,S,2023-05-16,09:00:07,2882,B,43.50,1
5,77,025,None,None,1,W0042,ROD,100000016063,S,2023-05-16,09:00:07,2882,B,43.50,1
6,78,025,None,None,1,W0042,ROD,100000016056,S,2023-05-16,09:00:07,2882,B,43.50,2
7,79,025,None,None,1,W0042,ROD,100000016062,S,2023-05-16,09:00:07,2882,B,43.50,4
8,57,025,None,None,1,W003b,ROD,100000000000.0,S,2023-05-05,09:00:00,2412,B,126.50,1
9,58,025,None,None,1,W003b,ROD,100000000000.0,S,2023-05-05,09:00:00,2412,B,126.50,1


In [ ]:
append_index_list = []

for index, row in df.iterrows():
    cur_df = df[df['']]
    cur_df = df[(df['security_type'] == row.security_type[0]) & (order['code'] == str(row.code)) & (order['order_type'] == row.order_type) & 
        (order['action'] == row.action) & (order['order_price'].astype('float') == row.order_price)]
    
    append_index_list.append(index)


In [238]:
order

,trader_id,order_id,security_type,order_time,code,order_type,action,order_qty,order_price,status,order_date,strategy
0,025,W003O,S,08:50:03,8048,ROD,B,1,54.1,NaN,2023-05-19,2
1,025,W003P,S,08:50:03,8048,ROD,B,1,48.15,NaN,2023-05-19,2
2,025,W003Q,S,08:50:03,8048,ROD,B,1,54.1,NaN,2023-05-19,2
3,025,W003R,S,08:50:04,8048,ROD,B,1,48.15,NaN,2023-05-19,2
4,025,W003S,S,08:50:04,8048,ROD,B,1,54.1,NaN,2023-05-19,2
5,025,W003T,S,08:50:04,8048,ROD,B,1,48.15,NaN,2023-05-19,2
6,025,W003U,S,08:50:04,8446,ROD,B,1,127,NaN,2023-05-19,2
7,025,W003V,S,08:50:04,8446,ROD,B,1,113.5,NaN,2023-05-19,2
8,025,00000,S,08:50:04,8426,ROD,B,1,69.9,特定證券管制交易－類別錯誤,2023-05-19,6
9,025,00000,S,08:50:04,8426,ROD,B,1,63.4,特定證券管制交易－類別錯誤,2023-05-19,6


In [128]:
cli = TSDBClient(
    host="128.110.25.99",
    port=5432,
    user="chiubj",
    password="bunnygood",
    db="accountdb"
)

In [27]:
strategy_df = cli.execute_query('''
SELECT * FROM dealer.strategy
''', 
    out_type='df')

In [87]:
trade_df = cli.execute_query('''
    select 
        t_st.name as 策略, 
        t_pos.code as 代碼,
        t_pos.position as 部位,
        t_pos.avg_price as 成本價,
        t_quote.close as 收盤價,
        (position * (close - avg_price))*1000 as 損益, 
        (close / avg_price - 1) * 100 as "損益(%)",
        t_pos.trade_date

    from (
        select 
            strategy, 
            code,
            trade_date,
            sum((case when action = 'B' then 1 else -1 end) * qty) as position,
            sum(price*qty) / sum(qty) as avg_price
        from dealer.trades
        group by strategy, code, trade_date
    )t_pos
left join public.quote_snapshots t_quote on t_pos.code = t_quote.code
left join dealer.strategy t_st on t_pos.strategy = t_st.id 

order by t_pos.strategy, t_pos.code
''', 
    out_type='df')

In [244]:
orders = cli.execute_query(f'''
SELECT * FROM dealer.orders
''', out_type='df')


In [88]:
status_list = []

for i in range(len(trade_df)):
    if date(datetime.now().year, datetime.now().month, datetime.now().day) == date(trade_df.iloc[i]['trade_date'].year, trade_df.iloc[i]['trade_date'].month, trade_df.iloc[i]['trade_date'].day):
        strategy_id = int(strategy_df[strategy_df['name'] == trade_df.iloc[i]['策略']]['id'])
        holding_period = int(strategy_df[strategy_df['name'] == trade_df.iloc[i]['策略']]['holding_period'])
        in_date = trade_df.iloc[i]['trade_date']
        out_date = datetime.date(xtai.sessions_window(pd.Timestamp(in_date.strftime("%Y-%m-%d")), holding_period)[-1])
        
        status_list.append([strategy_id, 'S', trade_df.iloc[i]['代碼'], trade_df.iloc[i]['成本價'], trade_df.iloc[i]['部位'], in_date, out_date])


In [90]:
cli.execute_values_df(df = pd.DataFrame(status_list, columns=['strategy', 'security_type', 'code', 'cost', 'qty', 'in_date', 'out_date']), table = 'dealer.status')

In [168]:
from utils import Config, check_folder_path_exist

config = Config()

In [159]:

cur_status = cli.execute_query(f'''
SELECT * FROM dealer.status where out_date = '{datetime.datetime.now().strftime("%Y-%m-%d")}'
''', out_type='df')


In [179]:
strategy_df = cli.execute_query('''
SELECT * FROM dealer.strategy
''', 
    out_type='df')

In [188]:
strategy = '法說會前主力蠢蠢欲動'
code = '2330'

In [184]:
order_low_ratio = strategy_df[strategy_df['name'] == ]['order_low_ratio'].values[0]

In [191]:
last_close = cli.execute_query(f'''
SELECT * FROM public.quote_snapshots where code = '{code}'
''', out_type='df')['close'][0]

In [203]:
_add_spread(last_close, -_spread_cnt((1 + order_low_ratio/100) * last_close, last_close))

492.5

In [197]:
from utils.spread import _add_spread, _spread_cnt

In [163]:

cur_status = cli.execute_query(f'''
SELECT * FROM dealer.status where out_date = '2023-05-23'
''', out_type='df')


In [164]:
cur_status

,strategy,security_type,code,cost,qty,in_date,out_date
0,1,S,1504,48.7,13,2023-05-09,2023-05-23
1,2,S,3479,119.0,2,2023-05-09,2023-05-23


In [175]:

counter = 1
sell_signal = []

for index, row in cur_status.iterrows():

    strategy_name = strategy_df[strategy_df['id'] == row.strategy]['name'].values[0]
    strategy_print_path = f"{config.signal_print_path}{strategy_name}"
    f_buy = open(f"{strategy_print_path}\Buy.log", 'a+')
    f_sell = open(f"{strategy_print_path}\Sell.log", 'a+')
    cur_contracts_df = cli.execute_query(f'''
        SELECT * FROM sino.contracts where code = '{row.code}'
        ''', out_type='df')
    
    if row.security_type == 'S': security_type = 'Stock' 
    if row.security_type == 'F': security_type = 'Futures'
    action = 'S' if row.qty > 0 else 'B'

    signal_list = [f'O{counter}', security_type, str(datetime.datetime.now().timestamp()), row.code, 'ROD', action, str(abs(row.qty)), '%.2f'%cur_contracts_df['limit_down'].values[0]]
    counter += 1

    if action == 'B':
        f_buy.write(','.join(signal_list) + '\n')
    elif action == 'S':
        f_sell.write(','.join(signal_list) + '\n')

    f_buy.close()
    f_sell.close()
    


In [174]:
signal_list

['O1', 'Stock', '1684133002.514638', '1504', 'ROD', 'S', 13, '40.95']

In [170]:
strategy_print_path

'C:\\proprietary\\print\\0    法說會前主力蠢蠢欲動\nName: name, dtype: object'

In [161]:
sell_signal

[]